In [36]:
import os
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
#%matplotlib inline

folder_root = 'C:/Users/rk401/Desktop/main_aug/'
#folder_root = '/home/user1/anaconda3/0.data/train/'
save_img_path = folder_root + 'images'
save_label_path = folder_root + 'labels'

def convert(size, box) :
    x1 = (box[1] - box[3] / 2) * size[0]
    x2 = (box[1] + box[3] / 2) * size[0]
    y1 = (box[2] - box[4] / 2) * size[1]
    y2 = (box[2] + box[4] / 2) * size[1]

    return [x1, y1, x2, y2]
    

def revert(size, box): # box : [x1,y1,x2,y2]
    dw = 1./size[0] 
    dh = 1./size[1] 
    x = (box[0] + box[2])/2.0 
    y = (box[1] + box[3])/2.0 
    w = box[2] - box[0] 
    h = box[3] - box[1] 
    x = x*dw 
    w = w*dw 
    y = y*dh 
    h = h*dh 
    
    return [x,y,w,h]


def readBound(imgname, label_index) :
    with open(folder_root + 'labels/main_' + str(label_index) + '.txt', 'r') as f :
        s = f.read()
    s = s.replace('\n', '')
    result = s.split(' ')
    for i in range(len(result)) :
        result[i] = float(result[i])
    return result


filepath = folder_root + 'images/'

def augimg(idx, imgname, label_index) :
        bound = readBound(imgname, label_index) # 상대좌표 (x_start, y_start, width, height)를 읽어옵니다.
        class_num = int(bound[0]) # class_num에 클래스를 저장합니다.
        image = imageio.imread(filepath+imgname+".jpg") # 이미지를 불러옵니다.
        size = image.shape # 불러온 이미지의 size를 저장합니다.
        point = convert(size, bound) # 상대좌표 (x_start, y_start, width, height) -> 절대좌표 (x_start, y_start, x_end, y_end)
        
        bbs = BoundingBoxesOnImage([
            BoundingBox(x1=point[0], y1=point[1], x2=point[2], y2=point[3])
        ], shape=image.shape)

        # 실제 Augmentation 인자를 추가하는 부분입니다. Documentation을 참고하여 원하는 인자를 선택해주세요!
        seq = iaa.Sequential([
            iaa.Affine(scale=(0.5, 1.5)),
            iaa.Affine(scale={"x": (0.5, 1.5), "y": (0.5, 1.5)}),
            iaa.Affine(rotate=(-45, 45)),
            iaa.Affine(shear=(-16, 16)),
        ])

        # 이미지에 대해 Augmentation을 진행합니다. Rotation과 같은 인자가 있다면 Bounding Box의 좌표 역시 바뀌어야 합니다.
        image_aug, bbs_aug = seq(image=image, bounding_boxes=bbs)
       
        before = bbs.bounding_boxes[0]
        after = bbs_aug.bounding_boxes[0]
        
        aug_box = [after.x1, after.y1, after.x2, after.y2]
        aug_box = revert(size, aug_box)
        aug_box = [class_num] + aug_box
        
        for i in range(1, len(aug_box)) :
            aug_box[i] = round(aug_box[i], 6)
        for i in range(len(aug_box)) :
            aug_box[i] = str(aug_box[i])
        
        # 증강된 이미지에 대한 상대적인 Bounding Box좌표(x_start, y_start, width, height)를 txt파일 형태로 저장합니다. 
        with open(f"{save_label_path}/{imgname}"+"_aug"+f"{idx}"+".txt", 'w') as f :
            f.writelines([aug_box[0], ' ', aug_box[1], ' ', aug_box[2] , ' ', aug_box[3], ' ', aug_box[4], '\n'])
        
        # 증강된 이미지를 저장합니다.
        imageio.imwrite(f"{save_img_path}/{imgname}"+"_aug"+f"{idx}"+".jpg", image_aug)






In [37]:
print(len(os.listdir()))

1


In [38]:
print(os.listdir())

['main_17.jpg']


In [39]:
os.chdir(save_img_path)

namelist = []
for i in range(len(os.listdir())):
    file_fullname = os.listdir()[i]
    file_name = os.path.splitext(file_fullname)
    namelist.append(file_name[0])

imgs =  namelist # file path 내에 있는 이미지의 이름입니다.


In [40]:
print(os.listdir())

['main_17.jpg']


In [41]:
print(namelist)

['main_17']


In [42]:
print(imgs)

['main_17']


In [43]:

for label_index in range(len(os.listdir())):
    for i in imgs :
        for j in range(25) : # file path내에 있는 각 이미지를 500장씩 증강합니다.
            augimg(j, i, label_index+17)

C:\Users\rk401\AppData\Local\Temp\ipykernel_8608\250341190.py:52: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filepath+imgname+".jpg") # 이미지를 불러옵니다.
